In [0]:
import pandas as pd
import numpy as np
import string

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/IRE_Major_Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
data = pd.read_csv(root_path+'OLIDv1.0/olid-training-v1.0.tsv',sep='\t')
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [4]:
print("Number of data points in train data", data.shape)
print('-'*50)
print("The attributes of data :", data.columns.values)

Number of data points in train data (13240, 5)
--------------------------------------------------
The attributes of data : ['id' 'tweet' 'subtask_a' 'subtask_b' 'subtask_c']


# Data Pre-Processing

### Printing some random tweets

In [5]:
# printing some random tweets
print(data['tweet'].values[0])
print("="*50)
print(data['tweet'].values[1])
print("="*50)
print(data['tweet'].values[2])
print("="*50)
print(data['tweet'].values[150])
print("="*50)
print(data['tweet'].values[1000])
print("="*50)
print(data['tweet'].values[1500])
print("="*50)
print(data['tweet'].values[2000])
print("="*50)
print(data['tweet'].values[5000])
print("="*50)
print(data['tweet'].values[10000])
print("="*50)
print(data['tweet'].values[13000])
print("="*50)

@USER She should ask a few native Americans what their take on this is.
@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL
Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge in the competitive marketplace. URL #Amazon #MAGA #KAG #CHINA #TCOT
@USER He is doing life tho😑
@USER Damn he fell victim to a little thing called TOS much like the ones on his own site URL URL
@USER mad bc you’re hot as fuck and they can’t have you 😭❤️
@USER @USER What difference does it make if the gun death was a “fight” or “random”! Holding parents accountable could be part of gun control legislation. Write you Congressional representatives and demand it! Until Congress deals w/ gun violence it will continue &amp; ALL Americans are at risk!
@USER @USER Picked the fuck up
@USER @USER Real conservatives are stepping up.
@USER @USER It now makes sense how gun control is barely an issue vs immigration.


### 1. Decontract

In [0]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", " will not ", phrase)
    phrase = re.sub(r"can\'t", " can not ", phrase)

    # general
    phrase = re.sub(r"n\'t", " not ", phrase)
    phrase = re.sub(r"\’re", " are ", phrase)
    phrase = re.sub(r"\’s", " is ", phrase)
    phrase = re.sub(r"\’d", " would ", phrase)
    phrase = re.sub(r"\’ll", " will ", phrase)
    phrase = re.sub(r"\’t", " not ", phrase)
    phrase = re.sub(r"\’ve", " have ", phrase)
    phrase = re.sub(r"\’m", " am ", phrase)
    return phrase

def remove_line_breaks(text):
  text = text.replace('\\r', ' ')
  text = text.replace('\\"', ' ')
  text = text.replace('\\n', ' ')
  return text

In [7]:
clean_tweet = data['tweet'].map(decontracted)
data['new_tweet'] = clean_tweet 
clean_tweet = data['new_tweet'].map(remove_line_breaks)
data['new_tweet'] = clean_tweet 
# print(sent)
# print("="*50)
data['new_tweet'].head()

0    @USER She should ask a few native Americans wh...
1    @USER @USER Go home you are  drunk!!! @USER #M...
2    Amazon is investigating Chinese employees who ...
3    @USER Someone should'veTaken" this piece of sh...
4    @USER @USER Obama wanted liberals &amp; illega...
Name: new_tweet, dtype: object

### 2. Count user mentions in the tweet

In [0]:
def get_user_mentions(text):
  return len(re.findall("@USER", text))

In [9]:
n_user_mentions = data['new_tweet'].map(get_user_mentions)
data['user_mentions'] = n_user_mentions
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c,new_tweet,user_mentions
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,@USER She should ask a few native Americans wh...,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,@USER @USER Go home you are drunk!!! @USER #M...,3
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,Amazon is investigating Chinese employees who ...,0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"@USER Someone should'veTaken"" this piece of sh...",1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,@USER @USER Obama wanted liberals &amp; illega...,2


In [0]:
def remove_user(text):
  return re.sub("@USER", '', text)

In [11]:
new_tweet = data['new_tweet'].map(remove_user)
data['new_tweet'] = new_tweet
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c,new_tweet,user_mentions
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,She should ask a few native Americans what th...,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,Go home you are drunk!!! #MAGA #Trump2020 ...,3
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,Amazon is investigating Chinese employees who ...,0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"Someone should'veTaken"" this piece of shit to...",1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,Obama wanted liberals &amp; illegals to move...,2


### 3. Count Number of hastags

In [0]:
def get_hash_tags(text):
  return len(re.findall("#", text))

In [13]:
n_hash_tags = data['new_tweet'].map(get_hash_tags)
data['n_hash_tags'] = n_hash_tags
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c,new_tweet,user_mentions,n_hash_tags
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,She should ask a few native Americans what th...,1,0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,Go home you are drunk!!! #MAGA #Trump2020 ...,3,2
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,Amazon is investigating Chinese employees who ...,0,5
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"Someone should'veTaken"" this piece of shit to...",1,0
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,Obama wanted liberals &amp; illegals to move...,2,0


### 4. Count urls in tweet

In [0]:
def get_url_count(text):
  return len(re.findall("URL",text))

In [15]:
n_urls = data['new_tweet'].map(get_url_count)
data['n_urls'] = n_urls
data.head()

,id,tweet,subtask_a,subtask_b,subtask_c,new_tweet,user_mentions,n_hash_tags,n_urls
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,She should ask a few native Americans what th...,1,0,0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,Go home you are drunk!!! #MAGA #Trump2020 ...,3,2,1
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,Amazon is investigating Chinese employees who ...,0,5,1
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"Someone should'veTaken"" this piece of shit to...",1,0,0
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,Obama wanted liberals &amp; illegals to move...,2,0,0


In [0]:
def get_emoji_count(text):
  return len(re.findall('[^\w\d!@#$%^&*()-+[{}:;"\',.\s?”“\-’]+', text))

In [17]:
n_emojis = data['new_tweet'].map(get_emoji_count)
data['n_emojis'] = n_emojis
df = data[['id','new_tweet','n_emojis']]
pd.set_option('max_colwidth', 120)
df.head()


,id,new_tweet,n_emojis
0,86426,She should ask a few native Americans what their take on this is.,0
1,90194,Go home you are drunk!!! #MAGA #Trump2020 👊🇺🇸👊 URL,1
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,0
3,62688,"Someone should'veTaken"" this piece of shit to a volcano. 😂""",1
4,43605,Obama wanted liberals &amp; illegals to move into red states,0


In [18]:
data = data[['id','tweet','new_tweet','user_mentions','n_hash_tags','n_urls','n_emojis', 'subtask_a' , 'subtask_b', 'subtask_c']]
data.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans what their take on this is.,She should ask a few native Americans what their take on this is.,1,0,0,0,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,Go home you are drunk!!! #MAGA #Trump2020 👊🇺🇸👊 URL,3,2,1,1,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,0,5,1,0,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""","Someone should'veTaken"" this piece of shit to a volcano. 😂""",1,0,0,1,OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,Obama wanted liberals &amp; illegals to move into red states,2,0,0,0,NOT,NaN,NaN


### 5. Perform data cleaning on tweets: case folding, removal of stop words and special characters.


In [0]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= { 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","url","user"}

def perform_cleaning(text):
  phrase = text.lower().strip()
  phrase = ' '.join(e for e in phrase.split() if e not in stopwords)
  phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)
  if not phrase:
    phrase = text.lower().strip()
  return phrase


In [20]:
cleaned_tweets = data['new_tweet'].map(perform_cleaning)
data['new_tweet'] = cleaned_tweets
data.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans what their take on this is.,she ask native americans their take is,1,0,0,0,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,go home you drunk maga trump2020,3,2,1,1,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,amazon investigating chinese employees selling internal data third party sellers looking edge competitive marketplac...,0,5,1,0,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""",someone should vetaken piece shit volcano,1,0,0,1,OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,obama wanted liberals amp illegals move red states,2,0,0,0,NOT,NaN,NaN


In [0]:
def length_tweet(text):
  return(len(text.lower().split()))

In [22]:
length_original_tweets = data['tweet'].map(length_tweet)
data['original_tweet_length'] = length_original_tweets
data.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c,original_tweet_length
0,86426,@USER She should ask a few native Americans what their take on this is.,she ask native americans their take is,1,0,0,0,OFF,UNT,NaN,14
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,go home you drunk maga trump2020,3,2,1,1,OFF,TIN,IND,11
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,amazon investigating chinese employees selling internal data third party sellers looking edge competitive marketplac...,0,5,1,0,NOT,NaN,NaN,27
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""",someone should vetaken piece shit volcano,1,0,0,1,OFF,UNT,NaN,11
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,obama wanted liberals amp illegals move red states,2,0,0,0,NOT,NaN,NaN,12


In [23]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def do_tokenize_lemmatize(text):
  lemmatizer = WordNetLemmatizer().lemmatize

  phrase = " ".join(lemmatizer(x,pos="v") for x in word_tokenize(text))
  if not phrase:
    phrase = text
  return phrase

In [25]:
cleaned_tweets = data['new_tweet'].map(do_tokenize_lemmatize)
data['new_tweet'] = cleaned_tweets
data.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c,original_tweet_length
0,86426,@USER She should ask a few native Americans what their take on this is.,she ask native americans their take be,1,0,0,0,OFF,UNT,NaN,14
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,go home you drink maga trump2020,3,2,1,1,OFF,TIN,IND,11
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,amazon investigate chinese employees sell internal data third party sellers look edge competitive marketplace amazon...,0,5,1,0,NOT,NaN,NaN,27
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""",someone should vetaken piece shit volcano,1,0,0,1,OFF,UNT,NaN,11
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,obama want liberals amp illegals move red state,2,0,0,0,NOT,NaN,NaN,12


In [26]:
print("Tweets after cleaning the data. We will use these tweets to create various text representations\n\n")
print(data['new_tweet'].values[0])
print("="*50)
print(data['new_tweet'].values[1])
print("="*50)
print(data['new_tweet'].values[2])
print("="*50)
print(data['new_tweet'].values[150])
print("="*50)
print(data['new_tweet'].values[1000])
print("="*50)
print(data['new_tweet'].values[1500])
print("="*50)
print(data['new_tweet'].values[2000])
print("="*50)
print(data['new_tweet'].values[5000])
print("="*50)
print(data['new_tweet'].values[10000])
print("="*50)
print(data['new_tweet'].values[13000])
print("="*50)

Tweets after cleaning the data. We will use these tweets to create various text representations


she ask native americans their take be
go home you drink maga trump2020
amazon investigate chinese employees sell internal data third party sellers look edge competitive marketplace amazon maga kag china tcot
he life tho
damn he fell victim little thing call tos much like ones his site
mad bc you hot fuck they not you
difference it make gun death fight random hold parent accountable could part gun control legislation write you congressional representatives demand it congress deal w gun violence it continue amp americans risk
pick fuck
real conservatives step up
it make sense gun control barely issue vs immigration


In [27]:
length_cleaned_tweets = data['new_tweet'].map(length_tweet)
data['new_tweet_length'] = length_cleaned_tweets
data.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c,original_tweet_length,new_tweet_length
0,86426,@USER She should ask a few native Americans what their take on this is.,she ask native americans their take be,1,0,0,0,OFF,UNT,NaN,14,7
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL,go home you drink maga trump2020,3,2,1,1,OFF,TIN,IND,11,6
2,16820,Amazon is investigating Chinese employees who are selling internal data to third-party sellers looking for an edge i...,amazon investigate chinese employees sell internal data third party sellers look edge competitive marketplace amazon...,0,5,1,0,NOT,NaN,NaN,27,19
3,62688,"@USER Someone should'veTaken"" this piece of shit to a volcano. 😂""",someone should vetaken piece shit volcano,1,0,0,1,OFF,UNT,NaN,11,6
4,43605,@USER @USER Obama wanted liberals &amp; illegals to move into red states,obama want liberals amp illegals move red state,2,0,0,0,NOT,NaN,NaN,12,8


In [0]:
data.to_csv(root_path+'preprocessed.csv', index=False) 